# Критерий Колмогорова - Смирнова для простой гипотезы

In [1]:
import random
import math
from math import e
import numpy as np
import pandas as pd

In [2]:
def make_expon_viborka(n, lam = 0.8):
    lst = []
    for i in range(n):
        r = random.random()
        x = -math.log(r,math.e) / lam
        lst.append(float('{:.3f}'.format(x)))
    
    return lst

In [3]:
n = 1000 # объем выборки

In [4]:
# Создаем 2 выборки 
viborka_1 = make_expon_viborka(n)
viborka_2 = make_expon_viborka(n)

In [5]:
def count_amount(viborka_1,viborka_2, inter = 1):
    
    int_min1 = min(viborka_1)
    int_min2 = min(viborka_2)
    int_max1 = max(viborka_1)
    int_max2 = max(viborka_2)
    
    my_max = max(int_max1, int_max2) # максимальное и минимальное граница значений
    my_min = min(int_min1, int_min2)
    

    lower_border = 0 # картка для подсчета количество вхожденией
    upper_border = inter
    
    interval = [] # интервал
    interval1 = [] 
    interval2 = []
    
    while lower_border <= my_max + inter:
        interval.append(lower_border)
        lower_border += inter
    
    interval1 = interval[:-1]
    interval2 = interval[1:]
    
    #print('Int', interval)
    amount1 = [] # количество вхождений чисел в каждый отрезок
    amount2 = []
    
    vib1 = sorted(viborka_1)
    vib2 = sorted(viborka_2)
    '''
    print('v1', vib1)
    print('9:', vib1[9])
    print(len(vib1))
    print('v2', vib2)
    '''
    count1 = 0
    count2 = 0
    
    
    #-------------------------------------------------------------------------------------
    lower_border = 0 # каретка для подсчета количество вхожденией
    upper_border = inter
    
    k  = 0 
    #print('!', interval[-1])
    while lower_border < interval[-1]:
        
        if k >= len(vib1):
            amount1.append(count1)
            lower_border = upper_border
            upper_border += inter
            count1 = 0
            
        elif lower_border <= vib1[k] < upper_border:
            #print(lower_border,' < ', vib1[k], '<', upper_border)
            count1 += 1
            k += 1

        else:
            lower_border = upper_border
            upper_border += inter
            amount1.append(count1)
            count1 = 0
        '''
        print('----')
        print('lower', lower_border)
        print('upper', upper_border)
        print('k =', k)
        '''
    
    lower_border = 0 # картка для подсчета количество вхожденией
    upper_border = inter
    
    k = 0
    
    while lower_border < interval[-1]:
        
        if k >= len(vib2):
            amount2.append(count2)
            lower_border = upper_border
            upper_border += inter
            count2 = 0
        elif lower_border <= vib2[k] < upper_border:
            #print(lower_border,' < ', vib2[k], '<', upper_border)
            count2 += 1
            k += 1

        else:
            lower_border = upper_border
            upper_border += inter
            amount2.append(count2)
            count2 = 0
    
    
    res = [amount1, amount2, interval1, interval2]
    
    return res

In [6]:
count_amount(viborka_1, viborka_2)

[[530, 246, 128, 51, 25, 13, 3, 3, 0, 0, 1],
 [528, 248, 117, 57, 19, 14, 10, 5, 1, 1, 0],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

In [7]:
am1, am2, inter1, inter2 = count_amount(viborka_1, viborka_2)
print(list(map(len, [am1, am2, inter1, inter2])))

[11, 11, 11, 11]


In [36]:
def table(viborka_1, viborka_2): 
    
    
    am1 = count_amount(viborka_1, viborka_2)[0]
    am2 = count_amount(viborka_1, viborka_2)[1]
    inter1 = count_amount(viborka_1, viborka_2)[2]
    inter2 = count_amount(viborka_1, viborka_2)[3]

    
    p = 0.2 # оценка парметра p
    
    print('Параметер равен: ', p)
    

    data = {'x_k': inter1,
           'x_k+1': inter2,
            'n_k_1': am1,
            'n_k_2': am2 }
    
    df = pd.DataFrame(data)
    n_im_list = []
    n_im_list_2 = []
    
    sum_k = 0 
    
    for k in range(len(am1)):
        sum_k += df.iloc[k]['n_k_1']
        n_im_list.append(sum_k)
    
    sum_k = 0 
    for k in range(len(am2)):
        sum_k += df.iloc[k]['n_k_2']
        n_im_list_2.append(sum_k)

    df['n_im_1'] = n_im_list
    df['n_im_2'] = n_im_list_2
    df['F1'] = df['n_im_1'] / df['n_k_1']
    df['F2'] = df['n_im_2'] / df['n_k_2']
    df['Module'] = abs(df['F1'] - df['F2'])
    df.loc['Сумма']= df.sum()
    #display(df)
    
    t = df
    k = 0
    
    for i in range(len(n_im_list)):
        if df.iloc[i]['n_k_1'] < 0.01 or df.iloc[i]['n_k_2'] < 0.01:
            #print('!')
            k += 1
            t = t.drop([i])


    t = t.drop(['Сумма'])
    t.loc['Сумма']= t.sum()

    
    display(t)
    #print(df['n_k'] - df['n_k_2'])
    #print(df['n_k_2'])
    print('Количество степеней свободы: %s = %s - 1 - %s '%(len(df) - 1 - k - 1 , len(df) - 1  , k) )  

    
    
    
    m = max(t['Module'][:-1])
    nm = math.sqrt( t.iloc[-1]['n_k_1'] * t.iloc[-1]['n_k_2'] / (t.iloc[-1]['n_k_1'] + t.iloc[-1]['n_k_2']))
    print('nm' , nm)
    print('Максимальное число: ', m)
    print('Значение критерия равно: ', np.around(m * nm, 4) )

In [37]:
table(viborka_1, viborka_2)

Параметер равен:  0.2


,x_k,x_k+1,n_k_1,n_k_2,n_im_1,n_im_2,F1,F2,Module
0,0.0,1.0,530.0,528.0,530.0,528.0,1.000000,1.000000,0.000000
1,1.0,2.0,246.0,248.0,776.0,776.0,3.154472,3.129032,0.025439
2,2.0,3.0,128.0,117.0,904.0,893.0,7.062500,7.632479,0.569979
3,3.0,4.0,51.0,57.0,955.0,950.0,18.725490,16.666667,2.058824
4,4.0,5.0,25.0,19.0,980.0,969.0,39.200000,51.000000,11.800000
5,5.0,6.0,13.0,14.0,993.0,983.0,76.384615,70.214286,6.170330
6,6.0,7.0,3.0,10.0,996.0,993.0,332.000000,99.300000,232.700000
7,7.0,8.0,3.0,5.0,999.0,998.0,333.000000,199.600000,133.400000
Сумма,28.0,36.0,999.0,998.0,7133.0,7090.0,810.527077,448.542463,386.724571


Количество степеней свободы: 7 = 11 - 1 - 3 
nm 22.343900170118427
Максимальное число:  232.7
Значение критерия равно:  5199.4256
